<div align = "center">

# **Limpieza extra para Frecuencia-Severidad**

</div>

## Librerias

In [14]:
library(dplyr)
library(tidyr)

## Data

In [15]:
datos_limpios <- read.csv("../data/processed/datos_limpios.csv")

In [16]:
head(datos_limpios)

,Amparo,SumaDePagos,Modelo,Color,Carroceria,Referencia1,Referencia2,CLASE_FASECOLDA,TIPO_VEHICULO,SERVICIO,Sexo_Aseg,Vr_Comercial,Pago,Accidentado,exposicion,mes,Edad_19_41,Edad_41_63,Edad_63_85
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,No aplica,0,2013,ROJO,SEDAN,VELOSTER,COUPE,AUTOMOVIL,Livianos,Particular,F,63430970,No,0,94,12,1,0,0
2,No aplica,0,2013,ROJO,SEDAN,VELOSTER,COUPE,AUTOMOVIL,Livianos,Particular,F,61150073,No,0,319,11,1,0,0
3,No aplica,0,2010,PLATA,HATCHBACK,ATOS [2],PRIME,AUTOMOVIL,Livianos,Particular,M,18415081,No,0,92,12,0,0,1
4,No aplica,0,2010,PLATA,HATCHBACK,ATOS [2],PRIME,AUTOMOVIL,Livianos,Particular,M,17752898,No,0,275,10,0,0,1
5,No aplica,0,2012,PLATA,SEDAN,ACCENT i25,1.4L,AUTOMOVIL,Livianos,Particular,F,37562534,No,0,322,12,1,0,0
6,No aplica,0,2012,PLATA,SEDAN,ACCENT i25,1.4L,AUTOMOVIL,Livianos,Particular,F,36667838,No,0,45,2,1,0,0


## Agrupando

In [17]:
# Variables categóricas relevantes para el modelo
variables_tarifa <- c("Modelo", "Color", "Carroceria", "CLASE_FASECOLDA", 
                      "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg")

# Crear variables de edad categóricas basadas en las dummies
datos_limpios <- datos_limpios %>%
  mutate(
    Grupo_Edad = case_when(
      Edad_19_41 == 1 ~ "19-41",
      Edad_41_63 == 1 ~ "41-63", 
      Edad_63_85 == 1 ~ "63-85",
      TRUE ~ "Otro"  # Por si hay casos fuera de estos rangos
    )
  )

# Agregar datos por celda tarifaria
tabla_agregada <- datos_limpios %>%
  group_by(across(all_of(c(variables_tarifa, "Grupo_Edad")))) %>%
  summarize(
    valor = sum(Vr_Comercial),

    # Para el modelo de frecuencia
    n_siniestros = sum(Accidentado),
    exposicion_total = sum(exposicion),
    
    # Para el modelo de severidad
    suma_pagos = sum(SumaDePagos),
    n_observaciones = n(),
    
    .groups = 'drop'
  )

# Convertir variables categóricas a factores en tabla_agregada
tabla_agregada <- tabla_agregada %>%
  mutate(
    Color = as.factor(Color),
    Carroceria = as.factor(Carroceria),
    CLASE_FASECOLDA = as.factor(CLASE_FASECOLDA),
    TIPO_VEHICULO = as.factor(TIPO_VEHICULO),
    SERVICIO = as.factor(SERVICIO),
    Sexo_Aseg = as.factor(Sexo_Aseg),
    Grupo_Edad = as.factor(Grupo_Edad)
  )

# Crear subset para severidad (solo celdas con siniestros)
tabla_severidad <- tabla_agregada %>%
  filter(n_siniestros > 0) %>%
  mutate(
    severidad_media = suma_pagos / n_siniestros,
    # Mantener todos los niveles de los factores aunque no aparezcan en los datos filtrados
    across(where(is.factor), ~factor(., levels = levels(.)))
  )

In [18]:
head(tabla_agregada)

Modelo,Color,Carroceria,CLASE_FASECOLDA,TIPO_VEHICULO,SERVICIO,Sexo_Aseg,Grupo_Edad,valor,n_siniestros,exposicion_total,suma_pagos,n_observaciones
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<dbl>,<int>
1993,ROJO,SEDAN,AUTOMOVIL,Livianos,Particular,F,41-63,64667906,2,1834,2558114,10
1993,ROJO,SEDAN,AUTOMOVIL,Livianos,Particular,M,19-41,76728037,2,1203,4883644,7
1995,VIOLETA,SEDAN,AUTOMOVIL,Livianos,Particular,F,19-41,207726695,0,3341,0,19
1997,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,41-63,121930553,2,3077,4238816,16
1998,AZUL,SEDAN,AUTOMOVIL,Livianos,Particular,F,19-41,130833256,2,2791,8213167,16
1998,AZUL,SEDAN,AUTOMOVIL,Livianos,Particular,F,41-63,71570700,0,1466,0,8


In [19]:
head(tabla_severidad)

Modelo,Color,Carroceria,CLASE_FASECOLDA,TIPO_VEHICULO,SERVICIO,Sexo_Aseg,Grupo_Edad,valor,n_siniestros,exposicion_total,suma_pagos,n_observaciones,severidad_media
<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>
1993,ROJO,SEDAN,AUTOMOVIL,Livianos,Particular,F,41-63,64667906,2,1834,2558114,10,1279057
1993,ROJO,SEDAN,AUTOMOVIL,Livianos,Particular,M,19-41,76728037,2,1203,4883644,7,2441822
1997,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,41-63,121930553,2,3077,4238816,16,2119408
1998,AZUL,SEDAN,AUTOMOVIL,Livianos,Particular,F,19-41,130833256,2,2791,8213167,16,4106584
1998,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,F,19-41,101517338,2,2410,8014069,13,4007035
1998,BLANCO,SEDAN,AUTOMOVIL,Livianos,Particular,M,19-41,180459141,4,4223,6450375,23,1612594


In [20]:
# Primero, veamos qué tipo de variables tienes
str(tabla_agregada)

tibble [449 × 13] (S3: tbl_df/tbl/data.frame)
 $ Modelo          : int [1:449] 1993 1993 1995 1997 1998 1998 1998 1998 1998 1998 ...
 $ Color           : Factor w/ 25 levels "AMARILLO","ARENA FINA",..: 19 19 25 5 3 3 5 5 5 11 ...
 $ Carroceria      : Factor w/ 11 levels "BUS-BUSETA","CABINADO",..: 9 9 9 9 9 9 9 9 9 9 ...
 $ CLASE_FASECOLDA : Factor w/ 8 levels "AUTOMOVIL","AUTOMOVIL TAXI",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ TIPO_VEHICULO   : Factor w/ 2 levels "Livianos","Pesados": 1 1 1 1 1 1 1 1 1 1 ...
 $ SERVICIO        : Factor w/ 3 levels "Particular","Publico",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Sexo_Aseg       : Factor w/ 2 levels "F","M": 1 2 1 2 1 1 1 1 2 2 ...
 $ Grupo_Edad      : Factor w/ 3 levels "19-41","41-63",..: 2 1 1 2 1 2 1 2 1 1 ...
 $ valor           : num [1:449] 6.47e+07 7.67e+07 2.08e+08 1.22e+08 1.31e+08 ...
 $ n_siniestros    : int [1:449] 2 2 0 2 2 0 2 0 4 0 ...
 $ exposicion_total: int [1:449] 1834 1203 3341 3077 2791 1466 2410 1467 4223 2934 ...
 $ suma_pagos      :

In [21]:
write.csv(tabla_severidad, "../data/processed/tabla_severidad.csv")